### Atualização da Criação e Execução de Janela para Controle de Estoque

##### Implementações para verificar e validar as inserções dos usuários e novas funcionalidades propostas por mim para um melhor controle de estoque

In [1]:
from tkinter import *
import pyodbc
from datetime import datetime

#Conexão com o Banco de Dados:

dados_conexao = ('Driver={SQLite3 ODBC Driver};'
                 'Server=localhost;'
                 'Database=Dados - Controle de Estoque/Estoque.db;')

conexao = pyodbc.connect(dados_conexao)
cursor = conexao.cursor()

#Funcionalidades do Sistema:
def registrar_dados_inseridos():
    nome = nome_insumo.get() or None
    qtde = qtde_insumo.get() or None
    data = data_insumo.get() or None
    lote = lote_insumo.get() or None
    return nome, qtde, data, lote
    
def validar_campos_obrigatorios(nome, qtde, data, lote):
    return all((nome, qtde, data, lote))

def validar_quantidade_insumo(qtde):
    try:
        quantidade = int(qtde)
        if quantidade <= 0:
            raise ValueError
        else:
            return True
    except ValueError:
        return False

def validar_lote(lote):
    try:
        lote = int(lote)
        if lote <= 0 or lote > 999:
            raise ValueError
        else:
            return True
    except ValueError:
        return False

def validar_data_insumo(data):
    data_validade = datetime.strptime(data, '%Y-%m-%d')
    if data_validade.date() < datetime.now().date():
        return False
    else:
        return True

def verificar_existência_insumo(nome, data, lote):
    cursor.execute(f'''
        SELECT Quantidade FROM Estoque 
        WHERE Produto = "{nome}" AND DataValidade = "{data}" AND Lote = "{lote}"
    ''')
    insumo_existente = cursor.fetchone()
    if insumo_existente:
        qtd_existente = insumo_existente[0]
        return int(qtd_existente)
    else:
        return None

def inserir_novo_insumo(nome, qtde, data, lote):
    cursor.execute(f'''
        INSERT INTO Estoque (Produto, Quantidade, DataValidade, Lote)
        VALUES
        ("{nome}", {qtde}, "{data}", {lote})
    ''')
    cursor.commit()

def limpar_campos():
    nome_insumo.delete(0, END)
    qtde_insumo.delete(0, END)
    data_insumo.delete(0, END)
    lote_insumo.delete(0, END)

def adicionar_insumo():
    nome, qtde, data, lote = registrar_dados_inseridos()
    
    if validar_campos_obrigatorios(nome, qtde, data, lote):
        pass
    else:
        caixa_texto.delete('1.0', END)
        caixa_texto.insert('1.0', 'Erro: Preencha todos os campos obrigatórios.')
        return
    
    if validar_quantidade_insumo(qtde):
        pass
    else:
        caixa_texto.delete('1.0', END)
        caixa_texto.insert('1.0', 'Erro: Quantidade inválida. Insira um número inteiro maior que zero.')
    
    if validar_data_insumo(data):
        pass
    else:
        caixa_texto.delete('1.0', END)
        caixa_texto.insert('1.0', 'Erro: Data de validade inválida ou anterior à data atual.')
        return
    
    if validar_lote(lote):
        pass
    else:
        caixa_texto.delete('1.0', END)
        caixa_texto.insert('1.0', 'Erro: Lote inválido. Insira um número de lote inteiro maior que 0 e menor que 999.')

    qtd_existente = verificar_existência_insumo(nome, data, lote)
    if qtd_existente is not None:
        nova_quantidade = qtd_existente + int(qtde)
        cursor.execute(f'''
            UPDATE Estoque 
            SET Quantidade = {nova_quantidade}
            WHERE Produto = "{nome}" AND DataValidade = "{data}" AND Lote = "{lote}"
        ''')
        cursor.commit()
        caixa_texto.delete('1.0', END)
        caixa_texto.insert('1.0', f'Sucesso: {qtde} unidade(s) de "{nome}" adicionado(s) ao lote {lote} existente!')
    else:
        inserir_novo_insumo(nome, qtde, data, lote)
        caixa_texto.delete('1.0', END)
        caixa_texto.insert('1.0', f'Sucesso: {qtde} unidade(s) de "{nome}" adicionado(s) ao lote {lote}!')
    
    limpar_campos()

def deletar_insumo():
    if len(nome_insumo.get()) < 2:
        # exibir uma mensagem -> nome insumo inválido
        # deletar tudo da caixa de texto
        caixa_texto.delete('1.0', END)
        # escrever na caixa de texto
        caixa_texto.insert('1.0', 'Nome do Insumo inválido!')
        # finalizar a função
        return
    
    # deletar o insumo
    cursor.execute(f'''
    DELETE FROM Estoque 
    WHERE Produto="{nome_insumo.get()}"
    ''')
    cursor.commit()
    
    # deletar tudo da caixa de texto
    caixa_texto.delete('1.0', END)
    
    # escrever na caixa de texto
    caixa_texto.insert('1.0', f'{nome_insumo.get()} deletado com sucesso!') 

def consumir_insumo():
    if len(nome_insumo.get()) < 2 or len(lote_insumo.get()) < 1 or len(qtde_insumo.get()) < 1:
        # exibir uma mensagem -> nome insumo inválido
        # deletar tudo da caixa de texto
        caixa_texto.delete('1.0', END)
        # escrever na caixa de texto
        caixa_texto.insert('1.0', f'Nome, Lote e Quantidade do Insumo inválidos!')
        # finalizar a função
        return
    
    # consumir o insumo
    cursor.execute(f'''
    UPDATE Estoque
    SET Quantidade=Quantidade-{qtde_insumo.get()}
    WHERE Produto="{nome_insumo.get()}" AND Lote={lote_insumo.get()}
    ''')
    cursor.commit()
    
    # deletar tudo da caixa de texto
    caixa_texto.delete('1.0', END)
    
    # escrever na caixa de texto
    caixa_texto.insert('1.0', f'{nome_insumo.get()} foi consumido em {qtde_insumo.get()} unidades!')

def visualizar_insumo():
    if len(nome_insumo.get()) < 2:
        # exibir uma mensagem -> nome insumo inválido
        # deletar tudo da caixa de texto
        caixa_texto.delete('1.0', END)
        # escrever na caixa de texto
        caixa_texto.insert('1.0', f'Nome do Insumo inválido!')
        # finalizar a função
        return
    
    # pesquisar pelo insumo
    cursor.execute(f'SELECT * FROM Estoque WHERE Produto="{nome_insumo.get()}"')
    valores = cursor.fetchall()

    texto = ""
    for id_produto, nome, quantidade, validade, lote in valores:
        texto = texto + f'''
        -----
        Produto: {nome}
        Quantidade: {quantidade}
        Validade: {validade}
        Lote: {lote}
        '''
    
    # deletar tudo da caixa de texto
    caixa_texto.delete('1.0', END)
    # escrever na caixa de texto
    caixa_texto.insert('1.0', texto)


#Criação da Janela:
    
window = Tk()

window.geometry('711x646')
window.configure(bg = '#ffffff')
canvas = Canvas(
    window,
    bg = '#ffffff',
    height = 646,
    width = 711,
    bd = 0,
    highlightthickness = 0,
    relief = 'ridge')
canvas.place(x = 0, y = 0)

background_img = PhotoImage(file = 'Janela - Controle de Estoque/background.png')
background = canvas.create_image(
    355.5, 323.0,
    image=background_img)

img0 = PhotoImage(file = 'Janela - Controle de Estoque/img0.png')
b0 = Button(
    image = img0,
    borderwidth = 0,
    highlightthickness = 0,
    command = visualizar_insumo,
    relief = 'flat')

b0.place(
    x = 479, y = 195,
    width = 178,
    height = 38)

img1 = PhotoImage(file = 'Janela - Controle de Estoque/img1.png')
b1 = Button(
    image = img1,
    borderwidth = 0,
    highlightthickness = 0,
    command = deletar_insumo,
    relief = 'flat')

b1.place(
    x = 247, y = 197,
    width = 178,
    height = 36)

img2 = PhotoImage(file = 'Janela - Controle de Estoque/img2.png')
b2 = Button(
    image = img2,
    borderwidth = 0,
    highlightthickness = 0,
    command = consumir_insumo,
    relief = 'flat')

b2.place(
    x = 479, y = 123,
    width = 178,
    height = 35)

img3 = PhotoImage(file = 'Janela - Controle de Estoque/img3.png')
b3 = Button(
    image = img3,
    borderwidth = 0,
    highlightthickness = 0,
    command = adicionar_insumo,
    relief = 'flat')

b3.place(
    x = 247, y = 125,
    width = 178,
    height = 34)

entry0_img = PhotoImage(file = 'Janela - Controle de Estoque/img_textBox0.png')
entry0_bg = canvas.create_image(
    455.0, 560.0,
    image = entry0_img)

caixa_texto = Text(
    bd = 0,
    bg = '#ffffff',
    highlightthickness = 0)

caixa_texto.place(
    x = 250, y = 502,
    width = 410,
    height = 114)

entry1_img = PhotoImage(file = 'Janela - Controle de Estoque/img_textBox1.png')
entry1_bg = canvas.create_image(
    517.0, 294.5,
    image = entry1_img)

nome_insumo = Entry(
    bd = 0,
    bg = '#ffffff',
    highlightthickness = 0)

nome_insumo.place(
    x = 377, y = 278,
    width = 280,
    height = 31)

entry2_img = PhotoImage(file = 'Janela - Controle de Estoque/img_textBox2.png')
entry2_bg = canvas.create_image(
    517.0, 340.5,
    image = entry2_img)

data_insumo = Entry(
    bd = 0,
    bg = '#ffffff',
    highlightthickness = 0)

data_insumo.place(
    x = 377, y = 324,
    width = 280,
    height = 31)

entry3_img = PhotoImage(file = 'Janela - Controle de Estoque/img_textBox3.png')
entry3_bg = canvas.create_image(
    517.0, 388.5,
    image = entry3_img)

lote_insumo = Entry(
    bd = 0,
    bg = '#ffffff',
    highlightthickness = 0)

lote_insumo.place(
    x = 377, y = 372,
    width = 280,
    height = 31)

entry4_img = PhotoImage(file = 'Janela - Controle de Estoque/img_textBox4.png')
entry4_bg = canvas.create_image(
    517.0, 436.5,
    image = entry4_img)

qtde_insumo = Entry(
    bd = 0,
    bg = '#ffffff',
    highlightthickness = 0)

qtde_insumo.place(
    x = 377, y = 420,
    width = 280,
    height = 31)

window.resizable(False, False)
window.mainloop()

cursor.close()
conexao.close()